In [34]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy, resample_apply
from backtesting.test import SMA
from backtesting import Backtest

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd 
import math as m
import os, glob

In [35]:
import warnings
warnings.filterwarnings("ignore")

In [36]:
pwd = os.getcwd()
datasource_path = os.path.join(pwd, r'../../../Datasets/Junio2020/procesados_classification/')
filenames = glob.glob(datasource_path + "/*.csv")
filenames.sort()
for file in filenames:
    print(os.path.basename(file))

C-EURUSD_FULL_1D_21F.csv
C-EURUSD_FULL_1H_25F.csv
C-EURUSD_FULL_1H_54F.csv


In [26]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
DATASET = pd.read_csv(filenames[0], header=0)
print(DATASET.head(20))
print(DATASET.shape)


      Gmt time     Open     High      Low    Close        Volume  Open_Bid  \
0   2004-01-01  1.25950  1.26052  1.24698  1.25733  4.621817e+05   1.25934   
1   2004-01-02  1.25717  1.26282  1.25214  1.25832  1.066578e+06   1.25701   
2   2004-01-04  1.25925  1.26487  1.25789  1.26318  6.879980e+04   1.25855   
3   2004-01-05  1.26324  1.26935  1.26301  1.26830  1.119689e+06   1.26308   
4   2004-01-06  1.26836  1.28093  1.26666  1.27150  1.125442e+06   1.26820   
5   2004-01-07  1.27161  1.27417  1.26180  1.26377  1.111343e+06   1.27145   
6   2004-01-08  1.26369  1.27854  1.25590  1.27696  1.115933e+06   1.26353   
7   2004-01-09  1.27698  1.28676  1.27034  1.28149  1.066698e+06   1.27682   
8   2004-01-11  1.28279  1.28572  1.28253  1.28477  7.567420e+04   1.28209   
9   2004-01-12  1.28498  1.28971  1.27327  1.27390  1.110720e+06   1.28482   
10  2004-01-13  1.27368  1.27957  1.27119  1.27836  1.114474e+06   1.27352   
11  2004-01-14  1.27850  1.27870  1.26296  1.26514  1.122805e+06

In [27]:
DATASET[['Close','Class']].head(10)

,Close,Class
0,1.25733,1.0
1,1.25832,2.0
2,1.26318,2.0
3,1.26830,1.0
4,1.27150,-2.0
5,1.26377,2.0
6,1.27696,2.0
7,1.28149,1.0
8,1.28477,-2.0
9,1.27390,2.0


In [28]:
DATASET['Position'] = DATASET['Class']
DATASET.loc[DATASET.Class == -2, 'Position'] = -1
DATASET.loc[DATASET.Class == 2, 'Position'] = 1
#DATASET.loc[DATASET.Class == -1, 'Position'] = 0 # remove Small movements
#DATASET.loc[DATASET.Class == 1, 'Position'] = 0 #Remove Small movements
DATASET[['Close','Position']].head(10)

,Close,Position
0,1.25733,1.0
1,1.25832,1.0
2,1.26318,1.0
3,1.26830,1.0
4,1.27150,-1.0
5,1.26377,1.0
6,1.27696,1.0
7,1.28149,1.0
8,1.28477,-1.0
9,1.27390,1.0


In [29]:
class Position(SignalStrategy):
    def init(self):
        global df
        super().init()  
        self.set_signal(self.data.Position)

In [31]:
DATASET['Class'].value_counts()

 2.0    1067
-2.0    1066
-1.0    1048
 1.0    1029
 0.0     927
Name: Class, dtype: int64

In [32]:
bt = Backtest(DATASET, Position, cash=1000, commission=.0001)
bt.run()

Start                               0
End                              5136
Duration                         5136
Exposure [%]                  99.9611
Equity Final [$]          1.34201e+11
Equity Peak [$]           1.34201e+11
Return [%]                1.34201e+10
Buy & Hold Return [%]          11.592
Max. Drawdown [%]            -1.19556
Avg. Drawdown [%]          -0.0722141
Max. Drawdown Duration              6
Avg. Drawdown Duration        2.24301
# Trades                         4208
Win Rate [%]                  97.6236
Best Trade [%]                3.36346
Worst Trade [%]             -0.809758
Avg. Trade [%]               0.443051
Max. Trade Duration                 5
Avg. Trade Duration           1.22006
Expectancy [%]               0.448805
SQN                            18.756
Sharpe Ratio                   1.1228
Sortino Ratio                 2.85365
Calmar Ratio                 0.370582
_strategy                    Position
dtype: object

In [33]:
bt.plot()